In [6]:
import sys
from pathlib import Path
wdir = Path.cwd().parent.absolute()
sys.path.insert(1, str(wdir))

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
import torch
import os

In [7]:
model = EEGNet(nb_classes = 5, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  51717


In [13]:
print(os.cpu_count())
print(torch.cuda.device_count()) 
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(device)
print(device.type == 'cuda')
if device.type == 'cuda':
    properties = torch.cuda.get_device_properties(device)
    print(properties)

16
1
cuda
True
_CudaDeviceProperties(name='Quadro P620', major=6, minor=1, total_memory=2047MB, multi_processor_count=4)
